In [39]:
# Import packages

import pandas as pd 
import numpy as np 
import os
from bs4 import BeautifulSoup
import time
import requests
import nfl_data_py as nfl
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

import pickle
from joblib import dump, load

import warnings

# Set certain settings for the notebook
pd.set_option("display.max_columns", None)

warnings.filterwarnings("ignore")

In [40]:
# Update before running

current_season = 2022
current_week = 12

In [41]:
# Get odds for this week's games 

# Manually Inputted
# While not ideal, manually inputting the odds in doesn't take too much time/effort
# In the future, want to create a web scraping bot to automate getting the necessary odds data
# For now, this will have to do

current_week_odds = pd.read_csv('../data/current_week_data.csv')
current_week_odds

,Unnamed: 0,season,week,away,home,away_moneyline,home_moneyline,away_spread,home_spread,over_under,home_full_name,away_full_name,home_qb,away_qb
0,2022124,2022,12,HOU,MIA,550,-750,13.5,-13.5,47.0,Miami Dolphins,Houston Texans,Tua Tagovailoa,Kyle Allen
1,2022125,2022,12,BAL,JAX,-195,165,-3.5,3.5,43.5,Jacksonville Jaguars,Baltimore Ravens,Trevor Lawrence,Lamar Jackson
2,2022126,2022,12,CHI,NYJ,215,-255,6.0,-6.0,38.5,New York Jets,Chicago Bears,Mike White,Trevor Siemian
3,2022127,2022,12,CIN,TEN,-150,130,-2.5,2.5,42.5,Tennessee Titans,Cincinnati Bengals,Ryan Tannehill,Joe Burrow
4,2022128,2022,12,ATL,WAS,170,-200,4.5,-4.5,40.5,Washington Football Team,Atlanta Falcons,Taylor Heinicke,Marcus Mariota
5,2022129,2022,12,DEN,CAR,-130,110,-1.5,1.5,36.0,Carolina Panthers,Denver Broncos,Baker Mayfield,Russell Wilson
6,20221210,2022,12,TB,CLE,-180,155,-3.5,3.5,42.0,Cleveland Browns,Tampa Bay Buccaneers,Jacoby Brissett,Tom Brady
7,20221211,2022,12,LV,SEA,165,-195,3.5,-3.5,48.0,Seattle Seahawks,Las Vegas Raiders,Geno Smith,Derek Carr
8,20221212,2022,12,LAC,ARI,-175,150,-3.0,3.0,48.0,Arizona Cardinals,Los Angeles Chargers,Kyler Murray,Justin Herbert
9,20221213,2022,12,LAR,KC,750,-1150,15.5,-15.5,42.0,Kansas City Chiefs,Los Angeles Rams,Patrick Mahomes,Matthew Stafford


In [42]:
# Get predictive stats dataframe 

predictive_stats = pd.read_csv('../data/predictive_stats_df.csv')
predictive_stats = predictive_stats[(predictive_stats.season == current_season) & (predictive_stats.week == current_week)]
predictive_stats

,Unnamed: 0,season,week,home,away,home_full_name,away_full_name,season_home,week_home,team_full_name_home,qb_home,total_pass_attempts_last_6_avg_home,total_passing_yards_last_6_avg_home,completions_last_6_avg_home,passing_epa_last_6_avg_home,pass_tds_last_6_avg_home,avg_cpoe_last_6_avg_home,qb_epa_last_6_avg_home,sacks_allowed_last_6_avg_home,interceptions_thrown_last_6_avg_home,qb_value_pre_last_6_avg_home,qb_adj_last_6_avg_home,total_rushes_last_6_avg_home,total_rush_yards_last_6_avg_home,rushing_epa_last_6_avg_home,rush_tds_last_6_avg_home,fumbles_last_6_avg_home,lost_fumbles_last_6_avg_home,ypc_last_6_avg_home,total_rushes_allowed_last_6_avg_home,total_rush_yards_allowed_last_6_avg_home,rush_tds_allowed_last_6_avg_home,total_pass_attempts_allowed_last_6_avg_home,passing_yards_allowed_last_6_avg_home,completions_allowed_last_6_avg_home,pass_tds_allowed_last_6_avg_home,avg_cpoe_allowed_last_6_avg_home,sacks_last_6_avg_home,fumbles_forced_last_6_avg_home,interceptions_last_6_avg_home,fumbles_recovered_last_6_avg_home,special_teams_epa_last_6_avg_home,passing_epa_def_last_6_avg_home,rushing_epa_def_last_6_avg_home,season_away,week_away,team_full_name_away,qb_away,total_pass_attempts_last_6_avg_away,total_passing_yards_last_6_avg_away,completions_last_6_avg_away,passing_epa_last_6_avg_away,pass_tds_last_6_avg_away,avg_cpoe_last_6_avg_away,qb_epa_last_6_avg_away,sacks_allowed_last_6_avg_away,interceptions_thrown_last_6_avg_away,qb_value_pre_last_6_avg_away,qb_adj_last_6_avg_away,total_rushes_last_6_avg_away,total_rush_yards_last_6_avg_away,rushing_epa_last_6_avg_away,rush_tds_last_6_avg_away,fumbles_last_6_avg_away,lost_fumbles_last_6_avg_away,ypc_last_6_avg_away,total_rushes_allowed_last_6_avg_away,total_rush_yards_allowed_last_6_avg_away,rush_tds_allowed_last_6_avg_away,total_pass_attempts_allowed_last_6_avg_away,passing_yards_allowed_last_6_avg_away,completions_allowed_last_6_avg_away,pass_tds_allowed_last_6_avg_away,avg_cpoe_allowed_last_6_avg_away,sacks_last_6_avg_away,fumbles_forced_last_6_avg_away,interceptions_last_6_avg_away,fumbles_recovered_last_6_avg_away,special_teams_epa_last_6_avg_away,passing_epa_def_last_6_avg_away,rushing_epa_def_last_6_avg_away
393,450,2022,12,JAX,BAL,Jacksonville Jaguars,Baltimore Ravens,2022,12,Jacksonville Jaguars,Trevor Lawrence,35.666667,231.333333,23.166667,1.681020,0.833333,1.473136,2.321464,1.833333,0.666667,139.191485,10.116718,27.333333,159.833333,2.368554,1.333333,0.666667,0.500000,5.791986,26.500000,121.666667,0.833333,35.500000,262.166667,23.666667,1.833333,0.329359,1.166667,0.500000,0.333333,0.333333,-0.522813,-7.798119,0.228077,2022,12,Baltimore Ravens,Lamar Jackson,28.666667,180.666667,18.000000,-1.928680,0.833333,-2.562786,0.943718,2.500000,0.500000,206.016505,14.115745,32.333333,177.500000,4.049504,1.166667,0.833333,0.333333,5.754960,20.166667,71.500000,0.833333,32.500000,232.000000,22.000000,0.833333,3.247331,3.833333,0.833333,0.666667,0.666667,2.715587,2.172194,0.817953
394,451,2022,12,NYJ,CHI,New York Jets,Chicago Bears,2022,12,New York Jets,Mike White,50.000000,378.000000,33.333333,4.984972,2.666667,-1.244552,5.065920,1.666667,2.333333,54.525189,-31.385853,25.666667,126.833333,1.358829,1.500000,0.333333,0.166667,4.786254,25.500000,111.333333,0.833333,36.833333,215.000000,22.666667,0.333333,-2.189345,4.000000,0.833333,0.833333,0.166667,0.976993,11.345410,-0.147303,2022,12,Chicago Bears,Trevor Siemian,38.250000,242.000000,21.750000,-3.227495,2.000000,-5.362166,-1.851666,1.750000,0.750000,97.258775,-60.796234,39.833333,232.000000,7.640986,1.500000,1.500000,0.166667,5.882217,25.500000,121.666667,1.666667,25.333333,202.666667,17.166667,1.333333,0.161081,1.166667,0.333333,0.666667,0.166667,-1.184675,-5.958727,0.594743
395,452,2022,12,TEN,CIN,Tennessee Titans,Cincinnati Bengals,2022,12,Tennessee Titans,Ryan Tannehill,26.333333,217.666667,17.833333,6.166570,1.500000,5.205069,4.725693,2.500000,0.333333,125.922447,-5.312514,31.500000,147.833333,-1.321499,1.166667,0.333333,0.333333,

In [43]:
# Load moneyline predictor algorithm 

ml_algo = load('../saved_models/xgboost_iter_1_depth_4_estimators_125.joblib')

In [44]:
# Load point spread predictor algorithm 

hs_algo = load('../saved_models/elasticnet_hs_iter_1_alpha_0.25_ratio_1.joblib')
as_algo = load('../saved_models/elasticnet_as_iter_1_alpha_1_ratio_0.1.joblib')

In [45]:
# Load over under predictor algorithm 

ou_algo = load('../saved_models/elasticnet_ts_iter_1_alpha_1_ratio_0.joblib')

In [46]:
# Function for getting predictions

def get_predictions(trained_model, testing_set, target = 'game_winner', predict_home = 'True'):
    
    test = testing_set.copy()
    test.drop(columns = ['season', 'week', 'home', 'away', 'home_full_name', 'away_full_name',
                         'season_home', 'week_home', 'season_away', 'week_away', 
                         'team_full_name_home', 'team_full_name_away', 'qb_home', 'qb_away', 'Unnamed: 0'], 
                          inplace=True)
    preds = testing_set.copy()
    preds = preds[['season', 'week', 'home_full_name', 'away_full_name']]
        
    if target == 'game_winner':
    
        pred = trained_model.predict(test)
        
        pred_prob = trained_model.predict_proba(test)
        
        preds['home_win_pred'] = pred
        preds[['away_win_prob', 'home_win_prob']] = pred_prob
        
        return preds
        
    elif target == 'point_spread':
        pred = trained_model.predict(test)
        
        if predict_home:
            preds['home_score_pred'] = pred
            
        else:
            preds['away_score_pred'] = pred
        
        return preds
    
    else:
        pred = trained_model.predict(test)
        
        preds['total_score_pred'] = pred
        
        return preds

In [47]:
# Get predictions

ml_preds = get_predictions(ml_algo, predictive_stats, target='game_winner')
hs_preds = get_predictions(hs_algo, predictive_stats, target='point_spread')
as_preds = get_predictions(as_algo, predictive_stats, target='point_spread', predict_home = False)
ts_preds = get_predictions(ou_algo, predictive_stats, target='total_score')

combined_predictions = ml_preds.merge(hs_preds).merge(as_preds).merge(ts_preds)
combined_predictions = combined_predictions[['season', 'week', 'home_full_name', 'away_full_name', 
                                            'home_win_pred', 'home_win_prob', 'away_win_prob', 
                                            'home_score_pred', 'away_score_pred', 'total_score_pred']]
combined_predictions

,season,week,home_full_name,away_full_name,home_win_pred,home_win_prob,away_win_prob,home_score_pred,away_score_pred,total_score_pred
0,2022,12,Jacksonville Jaguars,Baltimore Ravens,1,0.501564,0.498436,19.433399,24.407520,44.307330
1,2022,12,New York Jets,Chicago Bears,1,0.623600,0.376400,18.139717,24.113879,42.305366
2,2022,12,Tennessee Titans,Cincinnati Bengals,0,0.410819,0.589181,23.585550,21.301931,45.243097
3,2022,12,Washington Football Team,Atlanta Falcons,1,0.710235,0.289765,20.854725,16.812981,37.322525
4,2022,12,Carolina Panthers,Denver Broncos,1,0.596250,0.403750,20.414074,20.809704,41.035127
5,2022,12,Cleveland Browns,Tampa Bay Buccaneers,0,0.311658,0.688342,20.694577,25.645586,46.277703
6,2022,12,Seattle Seahawks,Las Vegas Raiders,1,0.664326,0.335674,27.048002,21.148958,48.360529
7,2022,12,Arizona Cardinals,Los Angeles Chargers,0,0.375029,0.624971,21.964420,22.543448,44.313383
8,2022,12,Kansas City Chiefs,Los Angeles Rams,1,0.869276,0.130724,29.683443,22.089007,51.257315
9,2022,12,San Francisco 49ers,New Orleans Saints,1,0.546895,0.453105,25.110866,21.176780,46.046392


In [48]:
# Add betting data

final_df = combined_predictions.merge(current_week_odds)
final_df.drop(columns = ['Unnamed: 0', 'away', 'home', 'home_qb', 'away_qb'], inplace=True)
final_df = final_df[['season', 'week', 'home_full_name', 'away_full_name', 
                                            'home_win_pred', 'home_win_prob', 'away_win_prob', 
                                            'home_score_pred', 'away_score_pred', 'total_score_pred',
                    'home_moneyline', 'away_moneyline', 'home_spread', 'away_spread', 'over_under']]
# final_df

In [49]:
# Add columns for Vegas moneyline implied win probabilities 

def get_implied_probability(game, open_line=True, home=True):
    
    if home:
        home_or_away = 'home'
    else:
        home_or_away = 'away'
        
    betting_column = '{}_moneyline'.format(home_or_away)
    
    ml_odds = game[betting_column]
    
    if ml_odds > 0:
        return (100/(ml_odds + 100))
    else:
        return (-ml_odds/(-ml_odds + 100))

final_df['implied_prob_home'] = final_df.apply(lambda x: get_implied_probability(x, home=True), axis=1)
final_df['implied_prob_away'] = final_df.apply(lambda x: get_implied_probability(x, home=False), axis=1)


In [50]:
# Add column to identify best algorithmic advantage
# Add column to calculate betting proportion amount

def get_prob_advantage_algo(game):
    advantage_home = game['home_win_prob'] - game['implied_prob_home']
    advantage_away = game['away_win_prob'] - game['implied_prob_away']
    
    if advantage_home > advantage_away:
        return advantage_home, 'home'
    else:
        return advantage_away, 'away'
    
def get_bet_proportional_algo(game):
    
    bet_weight = 0
    
    if game.best_prob_advantage >= .2 :
        bet_weight = 1
        
    return bet_weight
    
final_df = final_df.join(pd.DataFrame(final_df.apply(lambda x: get_prob_advantage_algo(x), axis=1).tolist(), columns=['best_prob_advantage', 'advantage_team']))
final_df['moneyline_bet_proportion'] = final_df.apply(lambda x: get_bet_proportional_algo(x), axis=1)

In [51]:
# Get spread differences 

final_df['home_spread_pred'] = final_df.apply(lambda x: x.away_score_pred - x.home_score_pred, axis=1)
final_df['away_spread_pred'] = final_df.apply(lambda x: x.home_score_pred - x.away_score_pred, axis=1)
final_df['home_line_diff'] = final_df.apply(lambda x: x.home_spread - x.home_spread_pred, axis=1)
final_df['away_line_diff'] = final_df.apply(lambda x: x.away_spread - x.away_spread_pred, axis=1)
final_df['total_score_diff'] = final_df.apply(lambda x: x.total_score_pred - x.over_under, axis=1)

In [52]:
# Get spread advantages, over/under advantages

def get_best_spread_advantage(game):
    
    if game.home_line_diff > game.away_line_diff:
        return game.home_line_diff, 'home'
    else:
        return game.away_line_diff, 'away'
    
def get_best_ou_advantage(game):
    
    if game.total_score_diff > 0:
        return 'over'
    else:
        return 'under'
    
def get_return_proportional_spread(game):
    
    bet_weight = 0
    
    if game.best_spread_advantage >= 5 :
        bet_weight = 1
        
    return bet_weight
    
def get_return_proportional_ou(game):
    
    bet_weight = 0
    
    if abs(game.total_score_diff) >= 3 :
        bet_weight = 1
        
    return bet_weight

def get_team_to_bet_on(game):
    
    if game.spread_bet_proportion <= 0:
        return game.advantage_team
    else:
        return game.spread_advantage_team

final_df[['best_spread_advantage', 'spread_advantage_team']] = final_df.apply(lambda x: get_best_spread_advantage(x), axis=1, result_type='expand')
final_df['spread_bet_proportion'] = final_df.apply(lambda x: get_return_proportional_spread(x), axis=1)
final_df['ou_advantage'] = final_df.apply(lambda x: get_best_ou_advantage(x), axis=1)
final_df['ou_bet_proportion'] = final_df.apply(lambda x: get_return_proportional_ou(x), axis=1)

In [53]:
final_df

,season,week,home_full_name,away_full_name,home_win_pred,home_win_prob,away_win_prob,home_score_pred,away_score_pred,total_score_pred,home_moneyline,away_moneyline,home_spread,away_spread,over_under,implied_prob_home,implied_prob_away,best_prob_advantage,advantage_team,moneyline_bet_proportion,home_spread_pred,away_spread_pred,home_line_diff,away_line_diff,total_score_diff,best_spread_advantage,spread_advantage_team,spread_bet_proportion,ou_advantage,ou_bet_proportion
0,2022,12,Jacksonville Jaguars,Baltimore Ravens,1,0.501564,0.498436,19.433399,24.407520,44.307330,165,-195,3.5,-3.5,43.5,0.377358,0.661017,0.124205,home,0,4.974121,-4.974121,-1.474121,1.474121,0.807330,1.474121,away,0,over,0
1,2022,12,New York Jets,Chicago Bears,1,0.623600,0.376400,18.139717,24.113879,42.305366,-255,215,-6.0,6.0,38.5,0.718310,0.317460,0.058939,away,0,5.974162,-5.974162,-11.974162,11.974162,3.805366,11.974162,away,1,over,1
2,2022,12,Tennessee Titans,Cincinnati Bengals,0,0.410819,0.589181,23.585550,21.301931,45.243097,130,-150,2.5,-2.5,42.5,0.434783,0.600000,-0.010819,away,0,-2.283619,2.283619,4.783619,-4.783619,2.743097,4.783619,home,0,over,0
3,2022,12,Washington Football Team,Atlanta Falcons,1,0.710235,0.289765,20.854725,16.812981,37.322525,-200,170,-4.5,4.5,40.5,0.666667,0.370370,0.043568,home,0,-4.041744,4.041744,-0.458256,0.458256,-3.177475,0.458256,away,0,under,1
4,2022,12,Carolina Panthers,Denver Broncos,1,0.596250,0.403750,20.414074,20.809704,41.035127,110,-130,1.5,-1.5,36.0,0.476190,0.565217,0.120060,home,0,0.395630,-0.395630,1.104370,-1.104370,5.035127,1.104370,home,0,over,1
5,2022,12,Cleveland Browns,Tampa Bay Buccaneers,0,0.311658,0.688342,20.694577,25.645586,46.277703,155,-180,3.5,-3.5,42.0,0.392157,0.642857,0.045484,away,0,4.951008,-4.951008,-1.451008,1.451008,4.277703,1.451008,away,0,over,1
6,2022,12,Seattle Seahawks,Las Vegas Raiders,1,0.664326,0.335674,27.048002,21.148958,48.360529,-195,165,-3.5,3.5,48.0,0.661017,0.377358,0.003309,home,0,-5.899045,5.899045,2.399045,-2.399045,0.360529,2.399045,home,0,over,0
7,2022,12,Arizona Cardinals,Los Angeles Chargers,0,0.375029,0.624971,21.964420,22.543448,44.313383,150,-175,3.0,-3.0,48.0,0.400000,0.636364,-0.011392,away,0,0.579028,-0.579028,2.420972,-2.420972,-3.686617,2.420972,home,0,under,1
8,2022,12,Kansas City Chiefs,Los Angeles Rams,1,0.869276,0.130724,29.683443,22.089007,51.257315,-1150,750,-15.5,15.5,42.0,0.920000,0.117647,0.013077,away,0,-7.594436,7.594436,-7.905564,7.905564,9.257315,7.905564,away,1,over,1
9,2022,12,San Francisco 49ers,New Orleans Saints,1,0.546895,0.453105,25.110866,21.176780,46.046392,-410,330,-9.5,9.5,43.5,0.803922,0.232558,0.220547,away,1,-3.934087,3.934087,-5.565913,5.565913,2.546392,5.565913,away,1,over,0


In [54]:
# Get wager amounts

base_wager = 100

gambling_df = final_df.copy()
gambling_df['ou_wager'] = gambling_df.ou_bet_proportion * base_wager
gambling_df['moneyline_wager'] = gambling_df.moneyline_bet_proportion * base_wager
gambling_df['spread_wager'] = gambling_df.spread_bet_proportion * base_wager
gambling_df = gambling_df[~((((gambling_df.spread_advantage_team == 'away') & (gambling_df.advantage_team == 'home')) 
                         | ((gambling_df.spread_advantage_team == 'home') & (gambling_df.advantage_team == 'away')))
                         & ((gambling_df.moneyline_wager>0) & (gambling_df.spread_wager>0)))]
gambling_df['wager_team'] = gambling_df.apply(lambda x: get_team_to_bet_on(x), axis=1)
gambling_df['total_wager'] = gambling_df.ou_wager + gambling_df.moneyline_wager + gambling_df.spread_wager
gambling_df

,season,week,home_full_name,away_full_name,home_win_pred,home_win_prob,away_win_prob,home_score_pred,away_score_pred,total_score_pred,home_moneyline,away_moneyline,home_spread,away_spread,over_under,implied_prob_home,implied_prob_away,best_prob_advantage,advantage_team,moneyline_bet_proportion,home_spread_pred,away_spread_pred,home_line_diff,away_line_diff,total_score_diff,best_spread_advantage,spread_advantage_team,spread_bet_proportion,ou_advantage,ou_bet_proportion,ou_wager,moneyline_wager,spread_wager,wager_team,total_wager
0,2022,12,Jacksonville Jaguars,Baltimore Ravens,1,0.501564,0.498436,19.433399,24.407520,44.307330,165,-195,3.5,-3.5,43.5,0.377358,0.661017,0.124205,home,0,4.974121,-4.974121,-1.474121,1.474121,0.807330,1.474121,away,0,over,0,0,0,0,home,0
1,2022,12,New York Jets,Chicago Bears,1,0.623600,0.376400,18.139717,24.113879,42.305366,-255,215,-6.0,6.0,38.5,0.718310,0.317460,0.058939,away,0,5.974162,-5.974162,-11.974162,11.974162,3.805366,11.974162,away,1,over,1,100,0,100,away,200
2,2022,12,Tennessee Titans,Cincinnati Bengals,0,0.410819,0.589181,23.585550,21.301931,45.243097,130,-150,2.5,-2.5,42.5,0.434783,0.600000,-0.010819,away,0,-2.283619,2.283619,4.783619,-4.783619,2.743097,4.783619,home,0,over,0,0,0,0,away,0
3,2022,12,Washington Football Team,Atlanta Falcons,1,0.710235,0.289765,20.854725,16.812981,37.322525,-200,170,-4.5,4.5,40.5,0.666667,0.370370,0.043568,home,0,-4.041744,4.041744,-0.458256,0.458256,-3.177475,0.458256,away,0,under,1,100,0,0,home,100
4,2022,12,Carolina Panthers,Denver Broncos,1,0.596250,0.403750,20.414074,20.809704,41.035127,110,-130,1.5,-1.5,36.0,0.476190,0.565217,0.120060,home,0,0.395630,-0.395630,1.104370,-1.104370,5.035127,1.104370,home,0,over,1,100,0,0,home,100
5,2022,12,Cleveland Browns,Tampa Bay Buccaneers,0,0.311658,0.688342,20.694577,25.645586,46.277703,155,-180,3.5,-3.5,42.0,0.392157,0.642857,0.045484,away,0,4.951008,-4.951008,-1.451008,1.451008,4.277703,1.451008,away,0,over,1,100,0,0,away,100
6,2022,12,Seattle Seahawks,Las Vegas Raiders,1,0.664326,0.335674,27.048002,21.148958,48.360529,-195,165,-3.5,3.5,48.0,0.661017,0.377358,0.003309,home,0,-5.899045,5.899045,2.399045,-2.399045,0.360529,2.399045,home,0,over,0,0,0,0,home,0
7,2022,12,Arizona Cardinals,Los Angeles Chargers,0,0.375029,0.624971,21.964420,22.543448,44.313383,150,-175,3.0,-3.0,48.0,0.400000,0.636364,-0.011392,away,0,0.579028,-0.579028,2.420972,-2.420972,-3.686617,2.420972,home,0,under,1,100,0,0,away,100
8,2022,12,Kansas City Chiefs,Los Angeles Rams,1,0.869276,0.130724,29.683443,22.089007,51.257315,-1150,750,-15.5,15.5,42.0,0.920000,0.117647,0.013077,away,0,-7.594436,7.594436,-7.905564,7.905564,9.257315,7.905564,away,1,over,1,100,0,100,away,200
9,2022,12,San Francisco 49ers,New Orleans Saints,1,0.546895,0.453105,25.110866,21.176780,46.046392,-410,330,-9.5,9.5,43.5,0.803922,0.232558,0.220547,away,1,-3.934087,3.934087,-5.565913,5.565913,2.546392,5.565913,away,1,over,0,0,100,100,away,200


In [55]:
# Total wager amount for the week

print('Week Total Wager Amount: {}'.format(sum(gambling_df.total_wager)))

Week Total Wager Amount: 1100


In [56]:
# Print bets

def print_bets(df):
    
    for index, row in df.iterrows():
        print('{} @ {}'.format(row.away_full_name, row.home_full_name))
        
        print('Advantage Team: {}, {}'.format(row.wager_team, row['{}_full_name'.format(row.wager_team)]))
        print('Over/Under Advantage: {}, {}'.format(row.ou_advantage, row.total_score_diff))
        print()
        if row.moneyline_wager > 0 :
            print('Moneyline Wager: {}, {} Dollars'.format(row['{}_full_name'.format(row.wager_team)], row.moneyline_wager))
        
        if row.spread_wager > 0 :
            print('Spread Wager: {}, {} Dollars'.format(row['{}_full_name'.format(row.wager_team)], row.spread_wager))
            
        if row.ou_wager > 0 :
            print('Over/Under Wager: {}, {} Dollars'.format(row.ou_advantage, row.ou_wager))
        
        print()
        print('Total Game Wager: {}'.format(row.total_wager))
        print()
        print()
    
    print('Week Total Wager Amount: {}'.format(sum(gambling_df.total_wager)))        

In [57]:
print_bets(gambling_df)

Baltimore Ravens @ Jacksonville Jaguars
Advantage Team: home, Jacksonville Jaguars
Over/Under Advantage: over, 0.8073304411459361


Total Game Wager: 0


Chicago Bears @ New York Jets
Advantage Team: away, Chicago Bears
Over/Under Advantage: over, 3.805366310844242

Spread Wager: Chicago Bears, 100 Dollars
Over/Under Wager: over, 100 Dollars

Total Game Wager: 200


Cincinnati Bengals @ Tennessee Titans
Advantage Team: away, Cincinnati Bengals
Over/Under Advantage: over, 2.743097196026781


Total Game Wager: 0


Atlanta Falcons @ Washington Football Team
Advantage Team: home, Washington Football Team
Over/Under Advantage: under, -3.177475391172436

Over/Under Wager: under, 100 Dollars

Total Game Wager: 100


Denver Broncos @ Carolina Panthers
Advantage Team: home, Carolina Panthers
Over/Under Advantage: over, 5.0351267838633476

Over/Under Wager: over, 100 Dollars

Total Game Wager: 100


Tampa Bay Buccaneers @ Cleveland Browns
Advantage Team: away, Tampa Bay Buccaneers
Over/Under Adv

In [39]:
# Analyze results

# Manual input of results
# Again not ideal and needs to be automated at some point, but good enough for now

# manual_input = {
#     '{}{}{}'.format(current_season, current_week, 1): [current_season, current_week, 'BUF','LAR',
#                                                       31, 10],
#     '{}{}{}'.format(current_season, current_week, 2): [current_season, current_week, 'BAL','NYJ',
#                                                       24, 9],
#     '{}{}{}'.format(current_season, current_week, 3): [current_season, current_week, 'SF','CHI',
#                                                       10, 19],
#     '{}{}{}'.format(current_season, current_week, 4): [current_season, current_week, 'IND','HOU',
#                                                       20, 20],
#     '{}{}{}'.format(current_season, current_week, 5): [current_season, current_week, 'PIT','CIN',
#                                                       23, 20],
#     '{}{}{}'.format(current_season, current_week, 6): [current_season, current_week, 'PHI','DET',
#                                                       38, 35],
#     '{}{}{}'.format(current_season, current_week, 7): [current_season, current_week, 'CLE','CAR',
#                                                       26, 24],
#     '{}{}{}'.format(current_season, current_week, 8): [current_season, current_week, 'NO','ATL',
#                                                       27, 26],
#     '{}{}{}'.format(current_season, current_week, 9): [current_season, current_week, 'NE','MIA',
#                                                       7, 20],
#     '{}{}{}'.format(current_season, current_week, 10): [current_season, current_week, 'JAX','WAS',
#                                                       22, 28],
#     '{}{}{}'.format(current_season, current_week, 11): [current_season, current_week, 'NYG','TEN',
#                                                       21, 20],
#     '{}{}{}'.format(current_season, current_week, 12): [current_season, current_week, 'KC','ARI',
#                                                       44, 21],
#     '{}{}{}'.format(current_season, current_week, 13): [current_season, current_week, 'GB','MIN',
#                                                       7, 23],
#     '{}{}{}'.format(current_season, current_week, 14): [current_season, current_week, 'LV','LAC',
#                                                       19, 24],
#     '{}{}{}'.format(current_season, current_week, 15): [current_season, current_week, 'TB','DAL',
#                                                       19, 3],
#     '{}{}{}'.format(current_season, current_week, 16): [current_season, current_week, 'DEN','SEA',
#                                                       16, 17]
# }

# def fix_team_names(team):
#     team_mapping = {
#         'ARI':'Arizona Cardinals',
#         'ATL':'Atlanta Falcons',
#         'BAL':'Baltimore Ravens',
#         'BUF':'Buffalo Bills',
#         'CAR':'Carolina Panthers',
#         'CHI':'Chicago Bears',
#         'CIN':'Cincinnati Bengals',
#         'CLE':'Cleveland Browns',
#         'DAL':'Dallas Cowboys',
#         'DEN':'Denver Broncos',
#         'DET':'Detroit Lions',
#         'GB':'Green Bay Packers',
#         'HOU':'Houston Texans',
#         'IND':'Indianapolis Colts',
#         'JAX':'Jacksonville Jaguars',
#         'KC':'Kansas City Chiefs',
#         'OAK':'Las Vegas Raiders',
#         'LV':'Las Vegas Raiders',
#         'LAC':'Los Angeles Chargers',
#         'LAR':'Los Angeles Rams',
#         'MIA':'Miami Dolphins',
#         'MIN':'Minnesota Vikings',
#         'NE':'New England Patriots',
#         'NO':'New Orleans Saints',
#         'NYG':'New York Giants',
#         'NYJ':'New York Jets',
#         'PHI':'Philadelphia Eagles',
#         'PIT':'Pittsburgh Steelers',
#         'SF':'San Francisco 49ers',
#         'SEA':'Seattle Seahawks',
#         'TB':'Tampa Bay Buccaneers',
#         'TEN':'Tennessee Titans',
#         'WAS':'Washington Football Team',
#         'WSH':'Washington Football Team'
#     }
    
#     return team_mapping[team]

# scores = pd.DataFrame.from_dict(manual_input, orient='index',
#                                           columns = ['season', 'week', 'away', 'home', 
#                                                     'away_score', 'home_score'])
# scores['home_full_name'] = scores.apply(lambda x: fix_team_names(x.home), axis=1)
# scores['away_full_name'] = scores.apply(lambda x: fix_team_names(x.away), axis=1)
# scores

In [65]:
# Join in gambling data and actual scores

def get_winner(game):
    if game.home_score > game.away_score:
        return 'home'
    elif game.away_score > game.home_score:
        return 'away'
    else:
        return 'push'
    
def get_spread_cover(game):
    if game.home_score + game.home_spread > game.away_score:
        return 'home'
    elif game.away_score + game.away_spread > game.home_score:
        return 'away'
    else:
        return 'push'
    
def get_ou_cover(game):
    if game.home_score + game.away_score > game.over_under:
        return 'over'
    if game.home_score + game.away_score < game.over_under:
        return 'under'
    else:
        return 'push'

results = gambling_df.merge(scores)
results['winner'] = results.apply(lambda x: get_winner(x), axis=1)
results['spread_cover'] = results.apply(lambda x: get_spread_cover(x), axis=1)
results['over_under_result'] = results.apply(lambda x: get_ou_cover(x), axis=1)
results

ValueError: Expected a 1D array, got an array with shape (0, 39)

In [ ]:
# Get results of bets

def get_moneyline_returns(game):
    if game.advantage_team == game.winner:
        if game['{}_moneyline'.format(game.advantage_team)] > 0:
            return np.round(((game['{}_moneyline'.format(game.advantage_team)]/100)*game.moneyline_wager), 2)
        else:
            return np.round(((100/abs(game['{}_moneyline'.format(game.advantage_team)]))*game.moneyline_wager), 2)
    elif game.winner == 'push':
        return 0
    else:
        return np.round(-game.moneyline_wager, 2)
    
def get_spread_returns(game):
    if game.advantage_team == game.spread_cover:
        return np.round(((100/110)*game.spread_wager), 2)
    elif game.winner == 'push':
        return 0
    else:
        return np.round(-game.spread_wager, 2)
    
def get_ou_returns(game):
    if game.ou_advantage == game.over_under_result:
        return np.round(((100/110)*game.ou_wager), 2)
    elif game.winner == 'push':
        return 0
    else:
        return np.round(-game.ou_wager, 2)
    
results['moneyline_returns'] = results.apply(lambda x: get_moneyline_returns(x), axis=1)
results['spread_returns'] = results.apply(lambda x: get_spread_returns(x), axis=1)
results['ou_returns'] = results.apply(lambda x: get_ou_returns(x), axis=1)
results

In [ ]:
# Analyze Results

print('Week Results')
print('Moneyline Returns: {}'.format(results.sum()['moneyline_returns']))
print('Spread Returns: {}'.format(results.sum()['spread_returns']))
print('Over/Under Returns: {}'.format(results.sum()['ou_returns']))
print('Total Returns: {}'.format(results.sum()['ou_returns'] + results.sum()['spread_returns'] + results.sum()['ou_returns']))

In [48]:
# Save predictions and bets

past_results = pd.read_csv('../data/past_results.csv')

past_results = past_results.append(results)
past_results = past_results.drop_duplicates()
past_results.to_csv('../data/past_results.csv')